# Evaluation metrics for Image Captioning tasks

**Denis Kokorev, Deepanshu Metha, and Pablo Valdunciel**

deko00002@stud.uni-saarland.de, 7007817  <br>
deme00001@stud.uni-saarland.de, 7011083 <br>
pava00001@stud.uni-saarland.de, 7010186 <br>

<hr>

This notebook introduces 4 of the most widespread evaluation metrics for Image Captioning tasks: 

1. BLEU 
2. METEOR
3. ROUGE-L
4. CIDEr 

It also includes examples of usage of some Python implementations of  these metrics.

## 1. BLEU (Bilingual Evaluation Understudy)
 

The [BLEU (Bilingual Evaluation Understudy)](https://www.aclweb.org/anthology/P02-1040.pdf) was originally thought as a metric to evaluate the quality of machine-translated text; however, it is actually the main metric in image captioning tasks. 

Given a a candidate sentence $c$ and a set of reference sentences $R =\{r_1, r_2, ..., r_m\}$, the *BLEU* metric for n-grams of size $n$, $w_n$, is:


$$BLEU_n(c, R) = \frac{\sum_{w_n \in c} min(Count_t(w_n), max_{j = \{1, ..., |R|\}} Count_{r_j}(w_n))}{\sum_{w_n \in c} Count_c(w_n)}$$

The $BLEU_n$ score will be in the range $[0,1]$, and the closer to 1 it is, the more similar the candidate sentence $c$ is to the reference sentences $R$.

The $BLEU_n$ score is usually calculated for $n=1,2,3,4$ and reported in a single $BLEU$ score combining the $BLEU_1$, $BLEU_2$, $BLEU_3$ and $BLEU_4$ scores in a weighted average: 

$$BLEU(c, R) = \sum_{n=1}^{N=4} \alpha_n BLEU_n(c, R)$$

being the uniform weights $\alpha_n = \frac{1}{4}$ the ones that are normally used.

#### Examples 

The nltk library provides an [implementation](https://www.nltk.org/api/nltk.translate.html#module-nltk.translate.bleu_score) of the *BLEU* score that is very easy to use. In this case, we will consider that the set of reference sentences contains a single sentence, $R = \{r\}$, and we will use the [sentence_bleu](https://www.nltk.org/api/nltk.translate.html#nltk.translate.bleu_score.sentence_bleu) function; however, another [corpus_bleu](https://www.nltk.org/api/nltk.translate.html#nltk.translate.bleu_score.corpus_bleu) function is also available.

In [2]:
from nltk.translate.bleu_score import sentence_bleu as bleu_score

Our reference and candidate sentences will be  the following:

In [3]:
references = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']]
candidate = ['the', 'fast', 'fox', 'jumped', 'over', 'the', 'lazy',  'dog']

We compute the individual $BLEU_n$ scores, for $n = 1,2,3,4$

In [4]:
bleu_1 = bleu_score(references, candidate, weights=(1, 0, 0, 0))
bleu_2 = bleu_score(references, candidate, weights=(0, 1, 0, 0))
bleu_3 = bleu_score(references, candidate, weights=(0, 0, 1, 0))
bleu_4 = bleu_score(references, candidate, weights=(0, 0, 0, 1))

print("BLEU_1 = ", bleu_1)
print("BLEU_2 = ", bleu_2)
print("BLEU_3 = ", bleu_3)
print("BLEU_4 = ", bleu_4)

BLEU_1 =  0.772184789761521
BLEU_2 =  0.6303549304175682
BLEU_3 =  0.5883312683897303
BLEU_4 =  0.5294981415507573


We can then compute the combine $BLEU$ score by combining the $BLEU_n$ scores or directly:

In [7]:
bleu_combined = (1/4)* (bleu_1 + bleu_2 + bleu_3 + bleu_4)

bleu = bleu_score(references, candidate, weights=(1/4, 1/4, 1/4, 1/4))

assert abs(bleu_combined - bleu) < 1e-2, 'incorrect calculation'

print("BLEU = ", bleu)

BLEU =  0.6240195441936914


If the sentences are identical, the *BLEU* score will be 1.0:

In [99]:
references = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']]
candidate = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
bleu_score(references, candidate, weights=(1/4, 1/4, 1/4, 1/4))

1.0

Changing a single word will already decrease it by nearly 0.25, even if the word is a synonym.

In [100]:
references = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']]
candidate = ['the', 'fast', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
bleu_score(references, candidate, weights=(1/4, 1/4, 1/4, 1/4))

0.7506238537503395

Simply removing the two adjectives for the *fox* and the *dog* will decrease it down to 0.5:

In [101]:
references = [['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']]
candidate = ['the', 'brown', 'fox', 'jumped', 'over', 'the', 'dog']
bleu_score(references, candidate, weights=(1/4, 1/4, 1/4, 1/4))

0.5025431541540227

## 2. METEOR (Metric for Evaluation of Translation with Explicit ORdering)

The [METEOR (Metric for Evaluation of Translation with Explicit ORdering)](https://www.researchgate.net/publication/228346240_METEOR_An_automatic_metric_for_MT_evaluation_with_high_levels_of_correlation_with_human_judgments) score is based on the harmonic mean of unigram precision and recall and it includes several features that aren't found in other metrics, such as **stemming** and **synonymy matching**. This metric seeks correlation at the sentence level, whereas *BLEU* seeks correlation at the corpus level.

In order to compute the METEOR score, it is necessary to compute the precision, $P$, and the recall, $R$, of the unigrams. Let $c$ be the candidate sentence and $r$ be the reference sentence:

$$P = \frac{m}{\sum_{w \in c} 1} = \frac{m}{|c|}$$

$$R = \frac{m}{\sum_{w \in r} 1}= \frac{m}{|r|}$$

where $m$ is the number of unigrams in the candidate sentence that are also found in the reference sentence.

Precision and recall are then combined using the harmonic mean in the following way:

$$F_{mean} = \frac{10 \cdot P \cdot R}{R + 9P}$$

In order to account for the congruity of larger n-grams, longer n-gram matches are used to compute a penalty $p$ for the alignment. The more mappings there are that are not adjacent in the reference and the candidate sentence, the higher the penalty will be.

In order to compute this penalty, unigrams are grouped into the fewest possible chunks, where a chunk is defined as a set of unigrams that are adjacent in the candidate and in the reference. The longer the adjacent mappings between the candidate and the reference, the fewer chunks there are. A translation that is identical to the reference will give just one chunk. The penalty p is computed as follows:

$$p = 0.5 (\frac{k}{m})^3$$

where $k$ is the number of chunks and $m$ is the number of unigrams that have been mapped.

The final score is calculated as 

$$M = F_{mean}(1-p)$$

The penalty $p$ will reduce the $F_{mean}$ up to 50% if there are no bigram or longer matches.

#### Examples

The Python nltk library also provides an [implementation](https://www.nltk.org/_modules/nltk/translate/meteor_score.html) for the *METEOR* score. Since the *METEOR* score makes use of *Wordnet*, it is necessary to download the package.

In [8]:
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/pabvald/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
from nltk.translate.meteor_score import single_meteor_score as meteor_score

We must notice the two main differences from the *bleu_score* implementation: 

1. The sentences are provided as a string, not as a list of tokens.
2. The order of the parameters is inverted, in this case the *candidate* sentence is provided before the *reference* sentence

In [10]:
reference = "the quick brown fox jumped over the lazy dog"
candidate = "the quick brown fox jumped over the lazy dog" 
meteor_score(candidate, reference)

0.9993141289437586

Changing the order of the words will obviously decrease the score:

In [11]:
reference = "the quick brown fox jumped over the lazy dog"
candidate = "the brown quick fox jumped over lazy the dog" 
meteor_score(candidate, reference)

0.7647462277091907

Since *METEOR* uses stemming and synonymy matching, the following changes do not decrese the score:

In [12]:
reference = "the quick brown fox jumped over the lazy dog"
candidate = "the fast brown fox jumps over the lazy dogs" 
meteor_score(candidate, reference)

0.9993141289437586

## 3. ROUGE-L

[ROUGE (Recall-Oriented Understudy for
Gisting Evaluation)](https://www.aclweb.org/anthology/W04-1013.pdf) includes measures to automatically determine the quality of a summary by
comparing it to other (ideal) summaries created by
humans. The measures count the number of overlapping units such as n-gram, word sequences, and
word pairs between the computer-generated summary to be evaluated and the ideal summaries created by humans. The measure that is used to measure the results in image captioning task is **ROUGE-L: Longest Common Subsequence**.

Let $c$ be the candidate sentence and $r$ be the reference sentence. The *ROUGE-L score* will be: 



$$ ROUGE_L(c, r) = 2 \cdot \frac{1}{\frac{1}{P} + \frac{1}{R}}$$ 

where: 
* $ P = \frac{LCS(c,r)}{|c|}$ is the precision

* $ R = \frac{LCS(c,r)}{|r|}$ is the recall 

* $LCS(c,r)$ is the longest common subsequence between sentences $c$ and $r


The $ROUGE_L$ score will be in the range $[0,1]$, and the closer to 1 it is, the more similar the candidate sentence $c$ is to the reference sentence $r$

#### Examples:

The Python [rouge_score](https://github.com/google-research/google-research/tree/master/rouge) package allows to compute the $ROUGE_L$ score  as follows:

In [3]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
score = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')['rougeL']

In [5]:
score

Score(precision=0.625, recall=0.5555555555555556, fmeasure=0.5882352941176471)

In [7]:
rouge_L = score.fmeasure
rouge_L

0.5882352941176471

We can see that the package provides with the final $ROUGE_L$ score (*fmeasure*) as well as the *precision* and *recall* used to compute it.

## 4. CIDEr 

[CIDEr (Consensus-based Image Description Evaluation)](https://arxiv.org/abs/1411.5726) was specifically design to evaluate image-captioning taskt. It gives more weightage to important n-grams and has a higher correlation with human consensus scores compared to metrics.

Let $c$ be the candidate sentence and $R =\{r_1, r_2, ..., r_k\}$ be a set of reference sentences. The *CIDEr score* considering n-grams of size $n$ is:


$$CIDEr_n(c, R) = \frac{1}{|R|} \sum_{j=1}^{k} \frac{g^n(c) \cdot g^n(r_j)}{||g^n(c)|| \cdot ||g^n(r_j)||}$$

where 

* $g^n(x)$ rerprsents the [TF-IDF](https://es.wikipedia.org/wiki/Tf-idf) scores of all n-grams in $x$

The scores from n-grams of varying lengths can be combined as follows: 

$$CIDEr(c, R) = \sum_{n=1}^{N} w_n \cdot CIDEr_n(c, R) $$

Empirically, it has been shown that uniform weights, $w_n = \frac{1}{n}$ work the best.

#### Examples 

There is not an implementation of *CIDEr* alone, but the [Microsoft COCO Caption Evaluation code]() implements several metrics including *BLEU* (1,2,3 and 4), *METEOR*, *ROUGE-L* and **CIDEr**. This code is included in the package [evaluation](./evaluation).

In [6]:
import json
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from evaluation.eval import eval


gts_path = '../evaluation/examples/gts.json' # ground-truths
res_path = '../evaluation/examples/res.json' # candidates

with open(gts_path, 'r') as f: 
    gts = json.load(f)
with open(res_path, 'r') as f:
    res = json.load(f)

mp = eval(gts, res, metrics=['bleu', 'cider', 'meteor', 'rouge'])
print(mp)

{'bleu': [0.579197412311677, 0.40424019147824525, 0.2784669955736699, 0.19079338759125988], 'cider': 0.6028573119212788, 'meteor': 0.19525467177780284, 'rouge': 0.39625269357570847}
